### Use CVXPY to upper bound process fidelity subject to known output state fidelities

In [32]:
import cvxpy as cp
import numpy as np

In [33]:
#First, we compute X_tar, the Choi matrix of the target gate
d = 4
U_tar = np.array([[1,0,0,0],[0,1,0,0],[0,0,np.sqrt(1/2),np.sqrt(1/2)],[0,0,np.sqrt(1/2),-np.sqrt(1/2)]]) # CH
phi_ME = np.zeros(d**2)
for j in range(d):
    phi_ME = phi_ME + np.kron(np.eye(1,d,j), np.eye(1,d,j))/np.sqrt(d)
rho_ME = np.outer(phi_ME, np.conj(phi_ME))
X_tar = np.kron(np.identity(d), U_tar) @ rho_ME @ np.kron(np.identity(d), np.conj(U_tar).T)

# basis states
state_dict = {'0':np.array([1,0]), '1':np.array([0,1]), '+':np.array([1,1])/np.sqrt(2), '-':np.array([1,-1])/np.sqrt(2), 'H':np.array([np.cos(np.pi/8), np.sin(np.pi/8)]), 'M':np.array([-np.sin(np.pi/8), np.cos(np.pi/8)]), 'i':np.array([1,1j])/np.sqrt(2), 'j':np.array([1,-1j])/np.sqrt(2)}

A = [] # constraint matrices
b = [] # constraint values

#  output state fidelities from physical CH experiment
fid_dict = {
    '00':1 - 0.00369,
    '01':1 - 0.00411,
    '10':1 - 0.00311,
    '11':1 - 0.00352,
    '+H':1 - 0.00397, 
    '+M': 1 - 0.00424,
    '-H': 1- 0.00352,
    '-M': 1 - 0.0038,
    '1+': 1 - 0.00329,
    '0-': 1 - 0.00397,
    '0+': 1 - 0.00374,
    '1-': 1 - 0.00385,
}

uncertainty_dict = {
    '00': 0.00019209372712298544,
    '01': 0.00028982753492378874,
    '10': 0.00017635192088548396,
    '11': 0.0001876166303929372,
    '+H': 0.00019924858845171275,
    '+M': 0.00020591260281974002,
    '-H': 0.0001876166303929372,
    '-M': 0.00019493588689617925,
    '1+': 0.00018138357147217052,
    '0-': 0.00019924858845171275,
    '0+': 0.00019339079605813715,
    '1-': 0.00019621416870348583,
}

# example of output state fidelities
out_dict = {
    '00': '00',
    '01': '01',
    '10': '1+',
    '11': '1-',
    '+H': '+H',
    '+M': '-M',
    '-H': '-H',
    '-M': '+M',
    '1+': '10',
    '0-': '01',
    '0+': '0+',
    '1-': '11',
}



povm_dict = {}
psi = state_dict['0']
psi_orth = state_dict['1']

povm0 = (1 - 0.00131)*np.outer(psi, np.conj(psi)) + (.00366)*np.outer(psi_orth, np.conj(psi_orth))
povm1 = (.00366)*np.outer(psi, np.conj(psi)) + (1 - 0.00131)*np.outer(psi_orth, np.conj(psi_orth))
povm_dict['0'] = povm0
povm_dict['1'] = povm1

psi = state_dict['+']
psi_orth = state_dict['-']
povm_plus = (1 - 0.00131)*np.outer(psi, np.conj(psi)) + (.00366)*np.outer(psi_orth, np.conj(psi_orth))
povm_minus = (.00366)*np.outer(psi, np.conj(psi)) + (1 - 0.00131)*np.outer(psi_orth, np.conj(psi_orth))

povm_dict['+'] = povm_plus
povm_dict['-'] = povm_minus


psi = state_dict['H']
psi_orth = state_dict['M']
povm_H = (1 - 0.00131)*np.outer(psi, np.conj(psi)) + (.00366)*np.outer(psi_orth, np.conj(psi_orth))
povm_M = (.00366)*np.outer(psi, np.conj(psi)) + (1 - 0.00131)*np.outer(psi_orth, np.conj(psi_orth))

povm_dict['H'] = povm_H
povm_dict['M'] = povm_M


for state in fid_dict:
    
    # ZZ or XX basis input states
    psi = np.kron(state_dict[state[0]], state_dict[state[1]])
    rho = np.outer(psi, np.conj(psi))

    M = np.kron(povm_dict[out_dict[state][0]],povm_dict[out_dict[state][1]])

    #M = U_tar @ rho @ np.conj(U_tar.T) # measurement operator
    A.append(np.kron(rho.T,M))
    b.append(fid_dict[state])
        

# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((d**2,d**2), hermitian=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]

# constraints on output state fidelities
constraints += [d*cp.real(cp.trace(A[k] @ X)) <= b[k] for k in range(len(A))]

# add partial trace constraints
for i in range(d):
    for j in range(d):
        B_ij = np.kron(np.outer(np.eye(1,d,i), np.eye(1,d,j)),np.identity(d))
        if i == j:
            constraints += [d*cp.real(cp.trace(B_ij @ X)) == 1.0]
        else:
            constraints += [d*cp.real(cp.trace(B_ij @ X)) == 0.0]


# setup and solve problem
prob = cp.Problem(cp.Maximize(cp.real(cp.trace(X_tar @ X))),
                  constraints)

F_lo = prob.solve()


print("Maximum process fidelity is ", round(F_lo,10))

Maximum process fidelity is  0.9987313158


Now we compute the derivative of the fidelity with respect to each of the state fidelities. For each state fidelity, we approximate the derivative by taking the finite quotient of the process fidelity at the endpoints of the state fidelity confidence interval.

In [3]:
derivative_dict = {}
for state in fid_dict.keys():
    fid_dict_new = fid_dict.copy()
    fid_dict_new[state] = fid_dict[state] + uncertainty_dict[state]
    
    b = [] # constraint values
    for state2 in fid_dict:
        b.append(fid_dict_new[state2])
            
    
    # Define and solve the CVXPY problem.
    # Create a symmetric matrix variable.
    X = cp.Variable((d**2,d**2), hermitian=True)
    # The operator >> denotes matrix inequality.
    constraints = [X >> 0]
    
    # constraints on output state fidelities
    constraints += [d*cp.real(cp.trace(A[k] @ X)) <= b[k] for k in range(len(A))]
    
    # add partial trace constraints
    for i in range(d):
        for j in range(d):
            B_ij = np.kron(np.outer(np.eye(1,d,i), np.eye(1,d,j)),np.identity(d))
            if i == j:
                constraints += [d*cp.real(cp.trace(B_ij @ X)) == 1.0]
            else:
                constraints += [d*cp.real(cp.trace(B_ij @ X)) == 0.0]
    
    
    
    # setup and solve problem
    prob = cp.Problem(cp.Maximize(cp.real(cp.trace(X_tar @ X))),
                      constraints)
    
    F_lo_plus = prob.solve()
    
    
    
    
    fid_dict_new[state] = fid_dict[state] - uncertainty_dict[state]
    b = [] # constraint values
    for state2 in fid_dict_new:
        b.append(fid_dict_new[state2])
            
    
    # Define and solve the CVXPY problem.
    # Create a symmetric matrix variable.
    X = cp.Variable((d**2,d**2), hermitian=True)
    # The operator >> denotes matrix inequality.
    constraints = [X >> 0]
    
    # constraints on output state fidelities
    constraints += [d*cp.real(cp.trace(A[k] @ X)) <= b[k] for k in range(len(A))]
    
    # add partial trace constraints
    for i in range(d):
        for j in range(d):
            B_ij = np.kron(np.outer(np.eye(1,d,i), np.eye(1,d,j)),np.identity(d))
            if i == j:
                constraints += [d*cp.real(cp.trace(B_ij @ X)) == 1.0]
            else:
                constraints += [d*cp.real(cp.trace(B_ij @ X)) == 0.0]
    
    
    # setup and solve problem
    prob = cp.Problem(cp.Maximize(cp.real(cp.trace(X_tar @ X))),
                      constraints)
    
    F_lo_minus = prob.solve()
    derivative_dict[state] = abs((F_lo_plus - F_lo_minus)/(2*uncertainty_dict[state]))

In [4]:
derivative_dict

{'00': 0.0001310412672407238,
 '01': 0.0022308852844037026,
 '10': 0.004062982297955246,
 '11': 9.162227988842575e-06,
 '+H': 0.25360849040963274,
 '+M': 0.2510972375746239,
 '-H': 0.24919635460089543,
 '-M': 0.24872288674205423,
 '1+': 0.0038851196697328953,
 '0-': 0.0035132247948572648,
 '0+': 0.0008056338133916894,
 '1-': 1.1405600902170672e-05}

Now we compute how the fidelity bound changes with respect to the measurement error

In [34]:
def compute_fidelity_bound(meas0_err, meas1_err):
    d = 4
    U_tar = np.array([[1,0,0,0],[0,1,0,0],[0,0,np.sqrt(1/2),np.sqrt(1/2)],[0,0,np.sqrt(1/2),-np.sqrt(1/2)]]) # CH
    phi_ME = np.zeros(d**2)
    for j in range(d):
        phi_ME = phi_ME + np.kron(np.eye(1,d,j), np.eye(1,d,j))/np.sqrt(d)
    rho_ME = np.outer(phi_ME, np.conj(phi_ME))
    X_tar = np.kron(np.identity(d), U_tar) @ rho_ME @ np.kron(np.identity(d), np.conj(U_tar).T)
    
    # basis states
    state_dict = {'0':np.array([1,0]), '1':np.array([0,1]), '+':np.array([1,1])/np.sqrt(2), '-':np.array([1,-1])/np.sqrt(2), 'H':np.array([np.cos(np.pi/8), np.sin(np.pi/8)]), 'M':np.array([-np.sin(np.pi/8), np.cos(np.pi/8)]), 'i':np.array([1,1j])/np.sqrt(2), 'j':np.array([1,-1j])/np.sqrt(2)}
    
    A = [] # constraint matrices
    b = [] # constraint values
    
    #  output state fidelities from physical CH experiment
    fid_dict = {
        '00':1 - 0.00369,
        '01':1 - 0.00411,
        '10':1 - 0.00311,
        '11':1 - 0.00352,
        '+H':1 - 0.00397, 
        '+M': 1 - 0.00424,
        '-H': 1- 0.00352,
        '-M': 1 - 0.0038,
        '1+': 1 - 0.00329,
        '0-': 1 - 0.00397,
        '0+': 1 - 0.00374,
        '1-': 1 - 0.00385,
    }
    
    uncertainty_dict = {
        '00': 0.00019209372712298544,
        '01': 0.00028982753492378874,
        '10': 0.00017635192088548396,
        '11': 0.0001876166303929372,
        '+H': 0.00019924858845171275,
        '+M': 0.00020591260281974002,
        '-H': 0.0001876166303929372,
        '-M': 0.00019493588689617925,
        '1+': 0.00018138357147217052,
        '0-': 0.00019924858845171275,
        '0+': 0.00019339079605813715,
        '1-': 0.00019621416870348583,
    }
    
    # example of output state fidelities
    out_dict = {
        '00': '00',
        '01': '01',
        '10': '1+',
        '11': '1-',
        '+H': '+H',
        '+M': '-M',
        '-H': '-H',
        '-M': '+M',
        '1+': '10',
        '0-': '01',
        '0+': '0+',
        '1-': '11',
    }
    
    
    
    povm_dict = {}
    psi = state_dict['0']
    psi_orth = state_dict['1']
    
    povm0 = (1 - meas0_err)*np.outer(psi, np.conj(psi)) + (meas1_err)*np.outer(psi_orth, np.conj(psi_orth))
    povm1 = (meas1_err)*np.outer(psi, np.conj(psi)) + (1 - meas0_err)*np.outer(psi_orth, np.conj(psi_orth))
    povm_dict['0'] = povm0
    povm_dict['1'] = povm1
    
    psi = state_dict['+']
    psi_orth = state_dict['-']
    povm_plus = (1 - meas0_err)*np.outer(psi, np.conj(psi)) + (meas1_err)*np.outer(psi_orth, np.conj(psi_orth))
    povm_minus = (meas1_err)*np.outer(psi, np.conj(psi)) + (1 - meas0_err)*np.outer(psi_orth, np.conj(psi_orth))
    
    povm_dict['+'] = povm_plus
    povm_dict['-'] = povm_minus
    
    
    psi = state_dict['H']
    psi_orth = state_dict['M']
    povm_H = (1 - meas0_err)*np.outer(psi, np.conj(psi)) + (meas1_err)*np.outer(psi_orth, np.conj(psi_orth))
    povm_M = (meas1_err)*np.outer(psi, np.conj(psi)) + (1 - meas0_err)*np.outer(psi_orth, np.conj(psi_orth))
    
    povm_dict['H'] = povm_H
    povm_dict['M'] = povm_M
    
    
    for state in fid_dict:
        
        # ZZ or XX basis input states
        psi = np.kron(state_dict[state[0]], state_dict[state[1]])
        rho = np.outer(psi, np.conj(psi))
    
        M = np.kron(povm_dict[out_dict[state][0]],povm_dict[out_dict[state][1]])
    
        #M = U_tar @ rho @ np.conj(U_tar.T) # measurement operator
        A.append(np.kron(rho.T,M))
        b.append(fid_dict[state])
            
    
    # Define and solve the CVXPY problem.
    # Create a symmetric matrix variable.
    X = cp.Variable((d**2,d**2), hermitian=True)
    # The operator >> denotes matrix inequality.
    constraints = [X >> 0]
    
    # constraints on output state fidelities
    constraints += [d*cp.real(cp.trace(A[k] @ X)) <= b[k] for k in range(len(A))]
    
    # add partial trace constraints
    for i in range(d):
        for j in range(d):
            B_ij = np.kron(np.outer(np.eye(1,d,i), np.eye(1,d,j)),np.identity(d))
            if i == j:
                constraints += [d*cp.real(cp.trace(B_ij @ X)) == 1.0]
            else:
                constraints += [d*cp.real(cp.trace(B_ij @ X)) == 0.0]
    
    
    # setup and solve problem
    prob = cp.Problem(cp.Maximize(cp.real(cp.trace(X_tar @ X))),
                      constraints)
    
    return(prob.solve())

Now we add the derivatives of the process fidelity with respect to the measurement error to our list. Again, these derivatives are computed by taking the difference at either endpoint of the confidence interval for the measurement error of 0 and 1. These confidence intervals have half-lengths of 3e-5 and 6e-5 respectively

In [35]:
derivative_dict['meas0_err'] = (compute_fidelity_bound(.00131 + .00003, 0.00366) - compute_fidelity_bound(.00131 - .00003, 0.00366))/.00006
derivative_dict['meas1_err'] = (compute_fidelity_bound(.00131, 0.00366 + .00006) - compute_fidelity_bound(.00131, 0.00366  - .00006))/.00012
uncertainty_dict['meas0_err'] = .00003
uncertainty_dict['meas1_err'] = .00006

We can view the derivative of the process fidelity with respect to each observable

In [36]:
derivative_dict

{'00': 0.0001310412672407238,
 '01': 0.0022308852844037026,
 '10': 0.004062982297955246,
 '11': 9.162227988842575e-06,
 '+H': 0.25360849040963274,
 '+M': 0.2510972375746239,
 '-H': 0.24919635460089543,
 '-M': 0.24872288674205423,
 '1+': 0.0038851196697328953,
 '0-': 0.0035132247948572648,
 '0+': 0.0008056338133916894,
 '1-': 1.1405600902170672e-05,
 'meas0_err': 2.0145945790010313,
 'meas1_err': -0.03298430007273959}

Using these derivatives, we can apply the propagation of uncertainty rule to estimate the total uncertainty of the process fidelity coming from each parameter

In [28]:
variance = 0
import math
for parameter in derivative_dict.keys():
    variance += (derivative_dict[parameter]*uncertainty_dict[parameter])**2
print(variance)
st_dev = math.sqrt(var_state)
print(st_dev)

1.3291090064129285e-08
0.00011528699000376966


We convert from process fidelity to average fidelity

In [38]:
print('average infidelity lower bound: ', 1- ((4*(F_lo) + 1)/(4+1)))
print('standard deviation of the average infidelity: ', (4/5)*(st_dev))

average infidelity lower bound:  0.001014947339189165
standard deviation of the average infidelity:  9.222959200301573e-05
